# 📊 Analisi Interattiva di Commenti Excel con AI

**Sistema ottimizzato per l'analisi qualitativa automatica di commenti con AI multi-provider**

## 🚀 Quick Start
1. **Configura**: Provider AI nel file `.env`
2. **Esegui**: Tutte le celle in ordine
3. **Seleziona**: Provider e file Excel
4. **Analizza**: Con template o prompt personalizzato
5. **Esporta**: Risultati in Excel

## ⚡ Funzionalità Principali
- **Multi-Provider**: Ollama (locale) + OpenRouter (cloud)
- **Batch Processing**: 5x più veloce
- **Template Intelligenti**: 6 tipi di analisi predefiniti
- **Interface Interattiva**: Configurazione guidata
- **Export Avanzato**: Report dettagliati

📚 **Documentazione completa**: `/docs/`

## 📚 1. Setup e Configurazione

In [ ]:
# Import librerie principali
import pandas as pd
import logging
import os
import time
from datetime import datetime
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import moduli utils
from utils.ai_clients import create_llm_instance, test_all_providers
from utils.config_manager import load_config, get_default_config
from utils.data_parsers import load_excel_file, get_column_info, validate_excel_file
from utils.batch_processor import process_comments_batch, estimate_batch_time

# Carica configurazione
load_dotenv()

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("analisi_log.log"),
        logging.StreamHandler()
    ]
)

print("✅ Setup completato!")
print("📁 Moduli utils caricati dalla directory /utils/")
print("📋 Documentazione disponibile in /docs/")
print("🧪 Test disponibili in /tests/")

## 🎛️ 2. Selezione Provider AI

In [ ]:
# Verifica provider disponibili
provider_status = test_all_providers()

# Widget per selezione provider
provider_options = []
if provider_status.get('ollama', False):
    provider_options.append(('🏠 Ollama (Locale/Gratuito)', 'ollama'))
if provider_status.get('openrouter', False):
    provider_options.append(('🌐 OpenRouter (API)', 'openrouter'))

if not provider_options:
    print("❌ Nessun provider configurato! Configura almeno uno in .env")
    provider_options = [('❌ Nessun provider', 'none')]

provider_dropdown = widgets.Dropdown(
    options=provider_options,
    description='Provider:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

apply_provider_button = widgets.Button(
    description="✅ Attiva Provider",
    button_style='success',
    layout=widgets.Layout(width='200px')
)

provider_output = widgets.Output()

def on_provider_apply(b):
    global llm, AI_PROVIDER
    with provider_output:
        clear_output()
        if provider_dropdown.value == 'none':
            print("❌ Configura un provider nel file .env")
            return
        
        try:
            AI_PROVIDER = provider_dropdown.value
            llm = create_llm_instance({'provider': AI_PROVIDER})
            print(f"✅ Provider {AI_PROVIDER.upper()} attivato con successo!")
        except Exception as e:
            print(f"❌ Errore attivazione provider: {e}")

apply_provider_button.on_click(on_provider_apply)

display(widgets.VBox([
    widgets.HTML("<h3>🎯 Seleziona Provider AI</h3>"),
    provider_dropdown,
    apply_provider_button,
    provider_output
]))

## 📁 3. Caricamento File Excel

In [ ]:
# Rileva file Excel nella directory
excel_files = [f for f in os.listdir('.') if f.endswith(('.xlsx', '.xls'))]

if not excel_files:
    excel_files = ['❌ Nessun file .xlsx trovato']

file_dropdown = widgets.Dropdown(
    options=excel_files,
    description='File Excel:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

custom_path = widgets.Text(
    placeholder='Oppure inserisci percorso personalizzato...',
    description='Percorso:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

load_button = widgets.Button(
    description="📂 Carica File",
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

column_dropdown = widgets.Dropdown(
    options=[],
    description='Colonna:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

confirm_column_button = widgets.Button(
    description="✅ Conferma Colonna",
    button_style='success',
    layout=widgets.Layout(width='200px')
)

file_output = widgets.Output()

def on_load_file(b):
    global df, file_path
    with file_output:
        clear_output()
        
        # Determina il percorso del file
        if custom_path.value.strip():
            file_path = custom_path.value.strip()
        else:
            if file_dropdown.value.startswith('❌'):
                print("❌ Nessun file selezionato")
                return
            file_path = file_dropdown.value
        
        try:
            # Carica e valida file
            df = load_excel_file(file_path)
            if df is None:
                print(f"❌ Impossibile caricare {file_path}")
                return
            
            # Aggiorna dropdown colonne
            columns = get_column_info(df)
            column_dropdown.options = list(columns.keys())
            
            print(f"✅ File caricato: {file_path}")
            print(f"📊 Dimensioni: {df.shape[0]} righe, {df.shape[1]} colonne")
            print(f"📋 Colonne disponibili: {list(columns.keys())}")
            
            # Mostra anteprima
            print("\n📋 Anteprima dati:")
            display(df.head())
            
        except Exception as e:
            print(f"❌ Errore caricamento file: {e}")

def on_confirm_column(b):
    global colonna_riferimento
    with file_output:
        if not column_dropdown.value:
            print("❌ Seleziona una colonna")
            return
        
        colonna_riferimento = column_dropdown.value
        
        # Analizza colonna selezionata
        non_vuoti = df[colonna_riferimento].notna().sum()
        vuoti = df[colonna_riferimento].isna().sum()
        
        print(f"\n✅ Colonna selezionata: {colonna_riferimento}")
        print(f"📊 Commenti validi: {non_vuoti}")
        print(f"📊 Celle vuote: {vuoti}")
        
        # Mostra esempi
        esempi = df[colonna_riferimento].dropna().head(3).tolist()
        print(f"\n📝 Esempi di commenti:")
        for i, esempio in enumerate(esempi, 1):
            print(f"   {i}. {esempio[:100]}{'...' if len(esempio) > 100 else ''}")

load_button.on_click(on_load_file)
confirm_column_button.on_click(on_confirm_column)

display(widgets.VBox([
    widgets.HTML("<h3>📁 Caricamento File Excel</h3>"),
    file_dropdown,
    custom_path,
    load_button,
    widgets.HTML("<br><h4>📋 Selezione Colonna Commenti</h4>"),
    column_dropdown,
    confirm_column_button,
    file_output
]))

## 🎯 4. Configurazione Prompt e Analisi

In [ ]:
# Template di prompt predefiniti
template_prompts = {
    "sentiment": {
        "nome": "📊 Sentiment Analysis",
        "prompt": """Analizza il sentiment di questi commenti e classifica in categorie.
Identifica: sentiment generale, intensità emotiva, emozioni specifiche.
Crea categorie di sentiment significative e descrittive.
Massimo 12 categorie."""
    },
    "feedback": {
        "nome": "💬 Feedback Prodotto",
        "prompt": """Analizza questi feedback di prodotto/servizio.
Identifica: aspetti positivi, negativi, suggerimenti, priorità.
Classifica per area di impatto e urgenza di intervento.
Massimo 15 categorie."""
    },
    "supporto": {
        "nome": "🎧 Supporto Clienti",
        "prompt": """Categorizza queste richieste di supporto.
Identifica: tipo problema, urgenza, area competenza, complessità.
Crea categorie per instradamento e prioritizzazione.
Massimo 15 categorie."""
    },
    "bug_report": {
        "nome": "🐛 Bug Reports",
        "prompt": """Analizza queste segnalazioni di bug.
Identifica: severità, area coinvolta, riproducibilità, impatto.
Classifica per priorità di risoluzione.
Massimo 12 categorie."""
    },
    "personalizzato": {
        "nome": "🎨 Analisi Personalizzata",
        "prompt": """Analizza questi commenti secondo i criteri specificati.
Crea categorie significative e descrittive.
Massimo 15 categorie."""
    }
}

# Widget per template
template_dropdown = widgets.Dropdown(
    options=[(info["nome"], key) for key, info in template_prompts.items()],
    description='Template:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

prompt_editor = widgets.Textarea(
    value=template_prompts["sentiment"]["prompt"],
    placeholder='Inserisci il tuo prompt personalizzato...',
    description='Prompt:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%', height='150px')
)

# Configurazione batch
batch_mode_check = widgets.Checkbox(
    value=True,
    description='⚡ Modalità Batch (5x più veloce)',
    style={'description_width': 'initial'}
)

batch_size_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    description='Batch Size:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

confirm_prompt_button = widgets.Button(
    description="✅ Conferma Configurazione",
    button_style='success',
    layout=widgets.Layout(width='250px')
)

prompt_output = widgets.Output()

def update_prompt_from_template(change):
    if change['type'] == 'change' and change['name'] == 'value':
        template_key = change['new']
        prompt_editor.value = template_prompts[template_key]["prompt"]

def on_confirm_prompt(b):
    global prompt_finale, BATCH_MODE, BATCH_SIZE
    with prompt_output:
        clear_output()
        
        prompt_finale = prompt_editor.value.strip()
        BATCH_MODE = batch_mode_check.value
        BATCH_SIZE = batch_size_slider.value
        
        if not prompt_finale:
            print("❌ Inserisci un prompt")
            return
        
        print("✅ Configurazione confermata:")
        print(f"📝 Template: {template_prompts[template_dropdown.value]['nome']}")
        print(f"⚡ Batch mode: {'Attivo' if BATCH_MODE else 'Disattivo'}")
        if BATCH_MODE:
            print(f"📦 Batch size: {BATCH_SIZE} commenti")
        print(f"📊 Prompt pronto per l'analisi ({len(prompt_finale)} caratteri)")

template_dropdown.observe(update_prompt_from_template)
confirm_prompt_button.on_click(on_confirm_prompt)

display(widgets.VBox([
    widgets.HTML("<h3>🎯 Configurazione Prompt</h3>"),
    template_dropdown,
    prompt_editor,
    widgets.HTML("<br><h4>⚡ Configurazione Performance</h4>"),
    batch_mode_check,
    batch_size_slider,
    confirm_prompt_button,
    prompt_output
]))

## 🚀 5. Esecuzione Analisi

In [ ]:
# Progress bar e controlli esecuzione
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progresso:',
    bar_style='info',
    style={'bar_color': '#2196F3'},
    layout=widgets.Layout(width='70%')
)

status_label = widgets.HTML(value="<b>⏳ Pronto per l'analisi</b>")

start_button = widgets.Button(
    description="🚀 Avvia Analisi",
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

analysis_output = widgets.Output()

def on_start_analysis(b):
    global risultati_analisi
    
    # Verifica prerequisiti
    if 'llm' not in globals():
        with analysis_output:
            print("❌ Attiva prima un provider AI")
        return
    
    if 'df' not in globals() or 'colonna_riferimento' not in globals():
        with analysis_output:
            print("❌ Carica prima un file Excel e seleziona la colonna")
        return
    
    if 'prompt_finale' not in globals():
        with analysis_output:
            print("❌ Configura prima il prompt")
        return
    
    with analysis_output:
        clear_output()
        
        try:
            # Preparazione dati
            comments = df[colonna_riferimento].dropna().tolist()
            total_comments = len(comments)
            
            print(f"🔍 Inizio analisi di {total_comments} commenti")
            print(f"⚡ Modalità: {'Batch' if BATCH_MODE else 'Singola'}")
            
            if BATCH_MODE:
                estimated_time = estimate_batch_time(total_comments, BATCH_SIZE)
                print(f"⏱️ Tempo stimato: {estimated_time:.1f} secondi")
            
            # Reset progress
            progress_bar.value = 0
            status_label.value = "<b>🔄 Analisi in corso...</b>"
            
            # Esecuzione analisi
            if BATCH_MODE:
                # Usa batch processor
                def update_progress(current, total):
                    progress = int((current / total) * 100)
                    progress_bar.value = progress
                    status_label.value = f"<b>⚡ Batch processing: {progress}% ({current}/{total})</b>"
                
                risultati_analisi = process_comments_batch(
                    comments, 
                    prompt_finale, 
                    llm, 
                    batch_size=BATCH_SIZE,
                    progress_callback=update_progress
                )
            else:
                # Processing singolo
                risultati_analisi = []
                for i, comment in enumerate(comments):
                    progress = int(((i + 1) / total_comments) * 100)
                    progress_bar.value = progress
                    status_label.value = f"<b>🔄 Processing: {progress}% ({i+1}/{total_comments})</b>"
                    
                    try:
                        response = llm.invoke(f"{prompt_finale}\n\nCommento: {comment}")
                        risultati_analisi.append(response)
                    except Exception as e:
                        print(f"⚠️ Errore commento {i+1}: {e}")
                        risultati_analisi.append("Errore di analisi")
                    
                    time.sleep(0.5)  # Pausa tra richieste
            
            # Completamento
            progress_bar.value = 100
            status_label.value = "<b>✅ Analisi completata!</b>"
            
            print(f"\n🎉 Analisi completata con successo!")
            print(f"📊 Commenti processati: {len(risultati_analisi)}")
            print(f"📝 Prime etichette: {risultati_analisi[:3]}")
            
        except Exception as e:
            progress_bar.value = 0
            status_label.value = "<b>❌ Errore analisi</b>"
            print(f"❌ Errore durante l'analisi: {e}")

start_button.on_click(on_start_analysis)

display(widgets.VBox([
    widgets.HTML("<h3>🚀 Esecuzione Analisi</h3>"),
    status_label,
    progress_bar,
    start_button,
    analysis_output
]))

## 📊 6. Visualizzazione Risultati ed Export

In [ ]:
# Widget per visualizzazione e export
view_results_button = widgets.Button(
    description="👁️ Visualizza Risultati",
    button_style='info',
    layout=widgets.Layout(width='200px')
)

export_button = widgets.Button(
    description="💾 Esporta Excel",
    button_style='success',
    layout=widgets.Layout(width='200px')
)

results_output = widgets.Output()

def on_view_results(b):
    with results_output:
        clear_output()
        
        if 'risultati_analisi' not in globals():
            print("❌ Esegui prima l'analisi")
            return
        
        # Crea DataFrame risultati
        df_results = df.copy()
        
        # Allinea risultati con DataFrame originale
        comments_series = df[colonna_riferimento].dropna()
        results_dict = {}
        
        for i, (idx, comment) in enumerate(comments_series.items()):
            if i < len(risultati_analisi):
                results_dict[idx] = risultati_analisi[i]
            else:
                results_dict[idx] = "Non analizzato"
        
        # Aggiungi colonna risultati
        df_results['Etichetta_AI'] = df_results.index.map(results_dict).fillna("Vuoto")
        
        print("📊 RISULTATI ANALISI")
        print("=" * 50)
        print(f"📝 Commenti totali: {len(df)}")
        print(f"✅ Commenti analizzati: {len(risultati_analisi)}")
        print(f"📋 Provider utilizzato: {AI_PROVIDER.upper()}")
        print(f"⚡ Modalità: {'Batch' if BATCH_MODE else 'Singola'}")
        
        # Statistiche etichette
        etichette_count = df_results['Etichetta_AI'].value_counts()
        print(f"\n🏷️ DISTRIBUZIONE ETICHETTE (Top 10):")
        for etichetta, count in etichette_count.head(10).items():
            percentage = (count / len(df_results)) * 100
            print(f"   {etichetta}: {count} ({percentage:.1f}%)")
        
        # Anteprima risultati
        print(f"\n📋 ANTEPRIMA RISULTATI:")
        cols_to_show = [colonna_riferimento, 'Etichetta_AI']
        display(df_results[cols_to_show].head(10))
        
        # Salva risultati globali per export
        globals()['df_risultati_finali'] = df_results

def on_export_results(b):
    with results_output:
        if 'df_risultati_finali' not in globals():
            print("❌ Visualizza prima i risultati")
            return
        
        try:
            # Genera nome file con timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            template_name = template_dropdown.value
            provider_name = AI_PROVIDER
            
            output_filename = f"analisi_{template_name}_{provider_name}_{timestamp}.xlsx"
            
            # Esporta con metadati
            with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
                # Sheet principale con risultati
                df_risultati_finali.to_excel(writer, sheet_name='Risultati', index=False)
                
                # Sheet con metadati
                metadata = {
                    'Parametro': [
                        'File originale', 'Colonna analizzata', 'Provider AI',
                        'Template utilizzato', 'Modalità processing', 
                        'Batch size', 'Data analisi', 'Commenti totali',
                        'Commenti analizzati'
                    ],
                    'Valore': [
                        file_path, colonna_riferimento, AI_PROVIDER.upper(),
                        template_prompts[template_dropdown.value]['nome'],
                        'Batch' if BATCH_MODE else 'Singola',
                        BATCH_SIZE if BATCH_MODE else 'N/A',
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        len(df), len(risultati_analisi)
                    ]
                }
                
                pd.DataFrame(metadata).to_excel(writer, sheet_name='Metadati', index=False)
                
                # Sheet con statistiche
                etichette_stats = df_risultati_finali['Etichetta_AI'].value_counts().reset_index()
                etichette_stats.columns = ['Etichetta', 'Frequenza']
                etichette_stats['Percentuale'] = (etichette_stats['Frequenza'] / len(df_risultati_finali) * 100).round(2)
                etichette_stats.to_excel(writer, sheet_name='Statistiche', index=False)
            
            print(f"✅ Risultati esportati in: {output_filename}")
            print(f"📊 File contiene {len(df_risultati_finali)} righe in 3 sheet")
            print(f"📋 Sheet: Risultati, Metadati, Statistiche")
            
        except Exception as e:
            print(f"❌ Errore durante l'export: {e}")

view_results_button.on_click(on_view_results)
export_button.on_click(on_export_results)

display(widgets.VBox([
    widgets.HTML("<h3>📊 Visualizzazione e Export</h3>"),
    widgets.HBox([view_results_button, export_button]),
    results_output
]))

## 🔧 7. Configurazione Avanzata (Opzionale)

In [ ]:
# Configurazioni avanzate per utenti esperti
advanced_config = widgets.Accordion([
    widgets.VBox([
        widgets.HTML("<h4>⚡ Parametri Batch Processing</h4>"),
        widgets.IntSlider(value=5, min=1, max=15, description='Max Batch Size:'),
        widgets.FloatSlider(value=1.0, min=0.1, max=5.0, description='Delay (sec):'),
        widgets.IntSlider(value=3, min=1, max=10, description='Max Retry:')
    ]),
    widgets.VBox([
        widgets.HTML("<h4>🎯 Parametri Qualità</h4>"),
        widgets.FloatSlider(value=0.7, min=0.1, max=1.0, description='Temperature:'),
        widgets.FloatSlider(value=0.3, min=0.0, max=1.0, description='Soglia Confidenza:'),
        widgets.IntSlider(value=150, min=50, max=500, description='Max Token:')
    ]),
    widgets.VBox([
        widgets.HTML("<h4>📊 Output e Logging</h4>"),
        widgets.Checkbox(value=True, description='Salva log dettagliato'),
        widgets.Checkbox(value=False, description='Debug mode'),
        widgets.Dropdown(options=['INFO', 'DEBUG', 'WARNING'], value='INFO', description='Log Level:')
    ])
])

advanced_config.set_title(0, '⚡ Performance')
advanced_config.set_title(1, '🎯 Qualità')
advanced_config.set_title(2, '📊 Output')

print("🔧 Configurazione avanzata disponibile (opzionale)")
display(advanced_config)

---

## 📚 Risorse Aggiuntive

- **📖 Documentazione**: Leggi `/docs/guida_utente.md` per la guida completa
- **⚙️ Configurazione**: Consulta `/docs/configurazione.md` per setup avanzato
- **🧪 Test**: Esegui `python tests/test_integration.py` per verificare il sistema
- **🔧 Utils**: Moduli riutilizzabili in `/utils/` per progetti personalizzati

### 🚀 Performance Tips
- Usa **Batch Mode** per file con molti commenti (5x più veloce)
- **Ollama** per privacy completa e uso intensivo
- **OpenRouter** per modelli all'avanguardia
- **Template** predefiniti per risultati rapidi e di qualità

### 🎯 Qualità Tips  
- **Prompt specifici** aumentano la precisione
- **Esempi concreti** nel prompt migliorano i risultati
- **Anteprima dati** per verificare la qualità del file
- **Validation** dei risultati prima dell'export finale